In [1]:

#%matplotlib inline
from random import random
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import seaborn as sn
import ppscore as pps
from datetime import datetime
from scipy.interpolate import *
import mpld3
%matplotlib inline
#%matplotlib notebook

In [2]:
def deleteOutlier(df, field):
    Q1 = np.percentile(df[field], 25, interpolation = 'midpoint')  
    Q2 = np.percentile(df[field], 50, interpolation = 'midpoint')  
    Q3 = np.percentile(df[field], 75, interpolation = 'midpoint')  
    IQR = Q3 - Q1  
    #print('Interquartile range is', IQR) 
    low_lim = Q1 - 1.5 * IQR 
    up_lim = Q3 + 1.5 * IQR 
    #print('low_limit is', low_lim) 
    #print('up_limit is', up_lim) 
    #Si los limites son distintos darle gas
    if low_lim!=up_lim:
        #df = df[(df['closing_price'] >= 99) & (df['closing_price'] <= 101)]
        #df = df[df['closing_price'].between(99, 101)]
        df = df[df[field].between(low_lim, up_lim)]
    
    return df

In [3]:
#mpld3.enable_notebook()
df = pd.read_csv("tita.csv")

In [4]:
#agrego tiempos
df['tstamp'] = pd.to_datetime(df['tstamp'])
df = df[df['tstamp'].astype('datetime64[ns]')>datetime(2020, 1, 1, 0, 0, 0)]
df = df[df['tstamp'].astype('datetime64[ns]')<datetime(2021, 3, 16, 0, 0, 0)]

In [5]:
varnames = ["CPU_C", 
            #"mirror_cell_C", 
            "temp_imo_c", 
            "nice_battery", 
            "pcm_3v3_v",
            "pcm_3v3_a",
            "pcm_5v_v",
            "pcm_5v_a",
            "sunvectorX",
            "sunvectorY",
            "sunvectorZ",
            'inEclipse',
            'elapsedTime',
            
            ]

In [6]:
#limpio outliers
print("Rows before outlier filter ", len(df.index));
for c in varnames:
  df = deleteOutlier(df, c)
print("Rows after outlier filter ", len(df.index));

Rows before outlier filter  62058
Rows after outlier filter  49451


In [7]:
X = df[['temp_imo_c', 'nice_battery']]
y = df['CPU_C']

In [8]:
%matplotlib widget
#plt.figure(figsize=(24,16), dpi= 100, facecolor='w', edgecolor='k')
fig = plt.figure(figsize=(12,8))
#fig.canvas.toolbar_visible = False
#fig.canvas.header_visible = False
#fig.canvas.resizable = True
ax = fig.add_subplot(projection='3d')

ax.scatter(X['temp_imo_c'], X['nice_battery'], y, marker='o')
ax.set_xlabel('temp_imo_c')
ax.set_ylabel('nice_battery')
ax.set_zlabel('CPU_C')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
#Se escalan las columnas de X
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
X_scaled = scale.fit_transform(X)
#p1 = np.polyfit(x, y,1)
#p2 = np.polyfit(x,y,2)
#p3 = np.polyfit(x,y,3)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, y, test_size=.2, random_state=0)

In [11]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2)
#realiza alguna transformacion del dataset
x_poly = poly.fit_transform(X_train)
poly.fit(X_train, Y_train)

PolynomialFeatures()

In [12]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x_poly, Y_train)

LinearRegression()

In [13]:
y_pred = model.predict(poly.fit_transform(X_test))

In [14]:
print(model.score(x_poly, Y_train))

0.8927203439061184


In [15]:
X_test

array([[ 0.39774771, -0.0266929 ],
       [-0.28395637, -0.25752909],
       [ 0.83155939,  1.013311  ],
       ...,
       [ 0.89353249,  0.9090624 ],
       [ 0.21182841, -0.94507344],
       [-0.43888912,  1.14734492]])

In [16]:

fig = plt.figure(figsize=(16, 6))
ax1 = fig.add_subplot(121, projection='3d')
surf = ax1.plot_surface(X_test['temp_imo_c'], X_test['nice_battery'], X_test.reshape(X.shape), 
                        rstride=1, cstride=1, cmap=cm.jet, alpha=0.5)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices